In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]

In [14]:
# TODO: Convert to use Azure Blob Storage
df = pd.read_parquet("/Users/larsankile/Downloads/openethdata_eth_data.parquet.gzip").sort_values("block_number")

# Filter out all transactions that are 'missed'
df = df[df.proposer_pubkey != "missed"]
df = df[~df.block_number.isna()]
df = df[~df.relay.isnull()]
df = df.astype({"block_number": int})
df = df.assign(mevboost_value=df.mevboost_value.astype(float) / 10**18)

print(df.shape)

df.head()

(2453440, 9)


,date,slot,block_number,relay,builder_pubkey,proposer_pubkey,mevboost_value,builder,validator
2030943,2022-09-15 08:33:47,4700567,15537940,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...,0.283204,Flashbots,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...
2241547,2022-09-15 08:35:23,4700575,15537948,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x9489e9354f515580f6a949f69eed311eeb26b268b744...,0.845802,Flashbots,0x9489e9354f515580f6a949f69eed311eeb26b268b744...
1700016,2022-09-15 08:41:35,4700606,15537978,flashbots,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,0x82d5ebc6c3ef5eda41dcbb1cbdfd5863ab6e4d16b137...,0.213524,Flashbots,stakewise
1923763,2022-09-15 08:59:11,4700694,15538064,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...,0.113136,Flashbots,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...
205280,2022-09-15 09:05:11,4700724,15538094,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xae1426c22c5c0d91d62c5a224759f44569a0a52e6cb8...,0.637987,Flashbots,rocketpool


In [15]:
# Convert the
df[df.mevboost_value.isnull()].shape

(0, 9)

In [16]:
df.to_sql("mev_boost", postgres_uri_mp, if_exists="replace", index=False, method="multi", chunksize=100_000)

2453440